In [1]:
import sys
print(sys.version) # 3.6.8

3.6.8 (default, Dec 24 2018, 19:24:27) 
[GCC 5.4.0 20160609]


In [2]:
import tensorflow as tf

tf.enable_eager_execution()

In [3]:
import matplotlib.pyplot as plt
from matplotlib.colors import DivergingNorm

import numpy as np
import pandas as pd

import h5py

import pickle
import cv2
import os

import PIL
import time
import pathlib

from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE

from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, r2_score

from pandas.plotting import scatter_matrix

from scipy.stats import gaussian_kde
from scipy.signal import find_peaks

from math import sqrt

from itertools import combinations, combinations_with_replacement

from cycler import cycler

In [4]:
BATCH_SIZE = 8

def _parse_predict_function(example_proto):
    # Create a description of the features.  
    feature_description = {
        # [AREA_UNIT, AREA_ENTRANCE, AREA_LDK, AREA_BEDROOM, AREA_BALCONY, AREA_BATHROOM]
        'floorplan': tf.FixedLenFeature([28, 28, 6], tf.float32, 
                                        default_value=tf.zeros([28, 28, 6], tf.float32)),
        
        'plan_id': tf.FixedLenFeature([], tf.string, default_value=''),
        'norm_year': tf.FixedLenFeature([], tf.float32, default_value=-1.0),
        'sido': tf.FixedLenFeature([], tf.int64, default_value=-1),
        'norm_area': tf.FixedLenFeature([], tf.float32, default_value=0.0),
        'is_area_85': tf.FixedLenFeature([], tf.int64, default_value=0),
        'is_core': tf.FixedLenFeature([], tf.int64, default_value=0),
        'num_rooms': tf.FixedLenFeature([], tf.int64, default_value=-1),
        'num_baths': tf.FixedLenFeature([], tf.int64, default_value=-1)
    }
    
    # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.parse_single_example(example_proto, feature_description)
    
    return parsed_example['floorplan'], parsed_example['plan_id'], parsed_example['norm_year']

def create_predict_dataset(filepaths):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepaths, compression_type="GZIP")
    
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_predict_function, num_parallel_calls=8)

    # Set the batchsize
    dataset = dataset.batch(BATCH_SIZE)
    
    return dataset

In [15]:
predict_dataset =  create_predict_dataset(
    ["/data/fp_train.tfrecord",
     "/data/fp_test.tfrecord"])

In [31]:
ids, pixels = [], []

iter = predict_dataset.make_one_shot_iterator()
for batch in iter:
    ids.extend(batch[1].numpy())
    pixels.extend(batch[0].numpy().sum(axis=(1,2)))
    
ids = [id.decode() for id in ids]

len(ids), ids[0], len(pixels), pixels[0].shape

(50540, '2829_105', 50540, (6,))

In [45]:
df = pd.DataFrame(dict(zip(ids, pixels))).T
df.columns = ['AREA_UNIT', 'AREA_ENTRANCE', 'AREA_LDK', 'AREA_BEDROOM', 'AREA_BALCONY', 'AREA_BATHROOM']
df = df.rename_axis(index='ID')
df

,AREA_UNIT,AREA_ENTRANCE,AREA_LDK,AREA_BEDROOM,AREA_BALCONY,AREA_BATHROOM
ID,,,,,,
2829_105,558.989929,5.676556,168.356445,179.012970,109.551949,11.366457
999_37,459.733459,10.718654,129.385895,173.962585,49.264877,17.427868
7770_55A,379.965271,9.728985,67.905609,173.404724,30.466793,9.850498
2140_92B,631.685181,7.549571,212.607193,218.043961,71.137909,15.703448
509_73,573.790405,11.936832,179.445786,174.572479,84.067726,16.123377
19478_113,679.496460,10.094253,170.109360,182.228699,142.364426,21.266609
8657_70,576.148621,11.317721,197.564728,184.694305,89.993294,20.063782
1856_61,570.955688,13.058487,121.789268,242.679932,94.112816,11.606352
25606_107C,556.480896,7.327203,132.050598,134.953537,192.912781,19.477461


In [46]:
df.to_csv('/data/fp_pixel.csv')